import library

In [8]:
import duckdb

In [9]:
con = duckdb.connect()
con.sql("INSTALL ducklake; LOAD ducklake;")

In [10]:
catalogMetadata = './catalog/catalog.ducklake'
managedStoredData = './managedStoreData/'

In [11]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS LocalLake (DATA_PATH '{managedStoredData}');")
con.sql("USE LocalLake;")

# **DEMONSTRASI**
##### **_UPDATE TABLE, ROLLBACK, SNAPSHOT, DAN EXPORT DATA DARI VERSION YANG DIINGINKAN_**

1. Check Snapshot Data dari Catalog ducklake (seperti commit history di `git`)

In [ ]:
con.sql(f"""
    SELECT 
        *
    FROM 
        ducklake_snapshots('LocalLake');
""")

┌─────────────┬───────────────────────────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                                        changes                                         │                       author                       │                   commit_message                   │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                                map(varchar, varchar[])                                 │                      varchar                       │                      varchar                       │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼────────────────────────────────────────────────────────────────────────────────────────┼───────

2. Menampilkan tabel jakarta di skema staging di versi skema yang diinginkan

In [ ]:
con.sql(f"""
    SELECT 
        *
    FROM 
        staging.dataJakarta
    AT (VERSION => 4);
""")


--- Time Travel ke Data Depok (Versi 0) ---


┌────────────────────────┬────────────────┬─────────────────────────┬───────────────────────┬──────────┬─────────┬───────────────────┬────────────────────┬───────────────────────────┬──────────────────────┬─────────────────────┬─────────────────────────────┬───────────────────────────┬───────────┬────────────────────────────────────────────┬──────────────────────────┬────────────┬──────────────────────┬─────────────────┬────────────────┬──────────────┬────────────────────┬──────────────────────────┬────────────────────────┬────────────┬───────────────────┬─────────┬────────────────────────────────────────┬─────────┬─────────────────────────┬────────────────────────┬────────────────────────┬─────────────────────────┬──────────────────────┬──────────┬─────────┬──────────┬────────┬────────┬────────────┬──────────┬──────────┬──────────────┬───────┬───────┬───────────┬───────┬───────┬──────────┬────────────────────┬───────────────────┬─────────────────────┬────────────────────┬─────────────

3. Melakukan `UPDATE` pada data tabel Jakarta dengan Nama Sekolah = `SDN DUREN SAWIT 01` atau NPSN = `20108557` dengan mengubah Provinsi nya menjadi **Jawa Tengah**

In [14]:
updateProvinsi = ("""
        UPDATE
            staging.dataJakarta
        SET 
            Provinsi = 'Jawa Tengah'
        WHERE
            NPSN = '20108557';
""")

con.execute("BEGIN;")
con.sql(updateProvinsi)
con.sql(f"CALL main.ducklake_set_commit_message('Locallake', 'Abdul', 'UPDATE: Set Provinsi Menjadi Jawa Tengah', extra_info => 'Update Pada NPSN 20108557')")
con.execute("COMMIT;")

4. Melakukan pengecekan apakah pada _latest data_ `UPDATE` berhasil dilakukan

In [15]:
con.sql("""
    SELECT Nama_Sekolah, Provinsi, NPSN
    FROM staging.dataJakarta
    WHERE NPSN = '20108557';
""")

┌────────────────────┬─────────────┬──────────┐
│    Nama_Sekolah    │  Provinsi   │   NPSN   │
│      varchar       │   varchar   │ varchar  │
├────────────────────┼─────────────┼──────────┤
│ SDN DUREN SAWIT 01 │ Jawa Tengah │ 20108557 │
└────────────────────┴─────────────┴──────────┘

5. Cek Snapshot nya apakah snapshot_id nya bertambah

In [16]:
con.sql("""SELECT * FROM ducklake_snapshots('Locallake')""")

┌─────────────┬───────────────────────────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬───────────────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                                        changes                                         │                       author                       │                   commit_message                   │     commit_extra_info     │
│    int64    │   timestamp with time zone    │     int64      │                                map(varchar, varchar[])                                 │                      varchar                       │                      varchar                       │          varchar          │
├─────────────┼───────────────────────────────┼────────────────┼────────────────────────────────────────────────────────────────────────

6. Setelah melakukan Demo karena ini hanyalah testing maka perlu di balikan ke versi asli nya

In [17]:
# Variabel Metadata Commit
author = 'Vincent'
commitMessage = 'Balikan data setelah demo UPDATE pada Time Travel.'
commitExtra = 'None.'

# Balikan Tabel Data Jakartanya ke versi awal (VERSION 2 = ketika tabel dataJakarta di buat)
rollbackQuery = f"""
    CREATE OR REPLACE TABLE staging.dataJakarta 
    AS SELECT * FROM staging.dataJakarta
    AT (VERSION => 2);
"""


con.execute("BEGIN;")
# Lakukan Rollback / Transaksi
con.sql(rollbackQuery)
# Set Metadata Commit nya (Pengubah, Pesan Commit, dan Info Tambahan)
con.sql(f"CALL main.ducklake_set_commit_message('Locallake', '{author}', '{commitMessage}', extra_info => '{commitExtra}')")
con.execute("COMMIT;")

In [18]:
con.sql("""
    SELECT Nama_Sekolah, Provinsi, NPSN
    FROM staging.dataJakarta
    WHERE NPSN = '20108557';
""")

┌────────────────────┬────────────────┬──────────┐
│    Nama_Sekolah    │    Provinsi    │   NPSN   │
│      varchar       │    varchar     │ varchar  │
├────────────────────┼────────────────┼──────────┤
│ SDN DUREN SAWIT 01 │ D.K.I. Jakarta │ 20108557 │
└────────────────────┴────────────────┴──────────┘

In [19]:
con.sql("""SELECT * FROM ducklake_snapshots('Locallake')""")

┌─────────────┬───────────────────────────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬────────────────────────────────────────────────────┬───────────────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                                        changes                                         │                       author                       │                   commit_message                   │     commit_extra_info     │
│    int64    │   timestamp with time zone    │     int64      │                                map(varchar, varchar[])                                 │                      varchar                       │                      varchar                       │          varchar          │
├─────────────┼───────────────────────────────┼────────────────┼────────────────────────────────────────────────────────────────────────

In [20]:
con.close()